In [2]:
import numpy as np
import pandas as pd
import math
import copy

In [3]:
# define function to download data
def load_data(url, filename):
    import urllib.request
    from zipfile import ZipFile
    
    response = urllib.request.urlretrieve(
        url,filename)
    #unzip
    with ZipFile(filename, 'r') as zip_ref:
        zip_ref.extractall()

In [4]:
# define fuction to read data from xml file
def import_data(filename):
    import xml.etree.ElementTree as et 
    xtree = et.parse(filename)
    xroot = xtree.getroot()
    
    return xroot;

In [5]:
# define function to create distance matrix
def dist_matrix(cities, xroot):
    #create distance matrix
    import numpy as np
    distance = np.zeros((cities,cities))
    
    #import data
    import xml.etree.ElementTree as et
    from_node = 0
    for child in xroot.iter('vertex'):
        for child1 in child:
            dist = float(child1.attrib.get('cost'))
            to_node = int(child1.text)
            distance[from_node, to_node] = dist

        from_node += 1
    
    max_distance = np.nanmax(distance)
    for i in range(cities):#
        distance[i,i] = max_distance*10 #very large number for distance to itself => no revisited 

    return distance 

In [6]:
# encoding a tour
def encode(detour):
    cities = len(detour)
    master = [x for x in range(cities)]
    entour = []
    
    for i in range(cities):
        entour.append(master.index(detour[i]))
        master.remove(detour[i])
    
    return entour

In [7]:
#decoding an individual
def decode(entour):
    cities = len(entour)    
    master = [x for x in range(cities)]
    detour = []
    
    for i in range(cities):
        detour.append(master[entour[i]])
        master.remove(master[entour[i]])
    
    return detour

In [8]:
# define function to swap 2 edges of nodes with index m and n (starting index 0)  
def tour_swap (tour, m, n):
    new_tour = copy.deepcopy(tour)
    new_tour[m:n+1] = reversed(new_tour[m:n+1])
    return new_tour

In [9]:
# define function to calculate the length of the tour 
def totalcost(distance, tour):
    length = 0
    for i in range(len(tour)-1):
        length += distance[tour[i]][tour[i+1]]
    return length

In [10]:
def two_opt (distance, tour):
    import copy
    
    best_tour = tour
    best_delta = 0
    improvement = True
    
    while improvement: 
        improvement = False
        for i in range(1,len(tour)-2):
            for j in range(i+1, len(tour)-1):
                temp_tour = tour_swap(tour, i, j)
                delta = totalcost(distance, tour[(i-1):(j+2)]) - totalcost(distance, temp_tour[(i-1):(j+2)]) #improvement = positive delta
                
                if delta > best_delta:
                    best_tour = copy.deepcopy(temp_tour)
                    best_delta = delta
                    improvement = True
    
    return (best_tour, totalcost(distance, best_tour))

In [11]:
#evaluate a tour
def evaluate(distance, tour, return_codetype, eval_two_opt = False):
    best_tour = tour
    if eval_two_opt:
        (best_tour, best_cost) = two_opt(distance, tour)
    else: #default
        best_cost = 0
        for i in range(len(tour)-1):
            best_cost += distance[tour[i]][tour[i+1]]
    
    best_cost +=  distance[tour[-1]][tour[0]] #return to starting point
    if return_codetype == 'encode':
        best_tour = encode(best_tour)
    
    return (best_tour, best_cost)

In [12]:
#format of a solution
def create_solution(distance, entour, codetype = 'permutation', create_two_opt = False):
    
    if codetype == 'encode':
        tour = decode(entour)
    elif codetype == 'permutation':
        tour = entour
    else:
        raise ValueError("Wrong codetype")
            
    
    (tour, best_cost) = evaluate(distance, tour, return_codetype = codetype, eval_two_opt = create_two_opt)
    # => return two_opt optimized tour & length if selecting two_opt option, else return original tour and its length
    # => return encode/permutation solution based on codetype
    
    solution = [best_cost, tour]
    
    return solution
    

In [13]:
# random initialize
def initialize(cities, distance, pop_size, codetype = 'permutation', init_two_opt = False):
    solution_pop = []
    entour_pop = [] #population of decision, 
                #in case of 2-opt include all original decisions, of which 2-opt optimized tour already included in solution population
    
    while len(entour_pop) < pop_size: #iterate to get pop_size candidates 
        entour = []
        
        if codetype == 'encode':    
            for i in range(cities):
                entour.append(np.random.randint(0,cities-i))  #position <= # of remaning cities
        elif codetype == 'permutation':
            entour = np.arange(cities)
            np.random.shuffle(entour)
            entour = list(entour)
        else:
            raise ValueError("Wrong codetype")            
        
        if entour not in entour_pop:
            entour_pop.append(entour)
            solution = create_solution(distance, entour, codetype, init_two_opt)
            if solution not in solution_pop: 
                solution_pop.append(solution)
    
    return solution_pop #return a population of solution candidates

In [14]:
# selection
def select(solution_pop, offspring_size, selecttype = 'fitness', tournament_size = None):
    offspring = []
    expected = np.mean([ sol[0] for sol in solution_pop])
    pop_size = len(solution_pop)
    f = [ (2*expected - solution_pop[i][0]) for i in range(pop_size)] #calculate f of each candidate
    sum_f = sum(f)

    if selecttype == 'tournament':
        #tournament size = # of solutions selected for each tournament
        if tournament_size is None: 
            tournament_size = len(solution_pop)//4
        
        for i in range(offspring_size):
            tournament = np.random.choice(np.arange(pop_size), size = tournament_size, replace = False) #random select k index
            tour_f = [f[i] for i in tournament]
            winner = tournament[np.argmax(tour_f)] #return index of winner
            offspring.append(solution_pop[winner])
            
        return offspring
    
    
    elif selecttype == 'rank': #rank selection
        f_index = [(value, idx) for (idx, value) in enumerate(f)]
        f_index = sorted(f_index, reverse = True) #sort based on value
        rank_index = [(idx, rank) for (rank, (value,idx)) in enumerate(f_index)] #get rank
        rank_index = sorted(rank_index) #sorted base on index
        rank = [r[1] for r in rank_index] #get a list of rank sorted on index
        
        eMax = 1 + np.random.random()
        eMin = 2 - eMax
        prob = [ (1/pop_size) * (eMax - (eMax - eMin)*(rank[i] - 1)/(pop_size-1)) for i in range(pop_size)] #individual prob
        
        prob_range = [ sum(prob[0:i+1]) for i in range(pop_size)] #cumulative prob of each candidate
        
    elif selecttype == 'fitness': #default selection mechanism
        prob_range = [ sum(f[0:i+1])/sum_f for i in range(pop_size)] #cumulative prob of each candidate
    
    else:
        raise ValueError("Wrong selecttype")        
    
    
    for i in range(offspring_size):
        p = np.random.random()

        for j in range(pop_size):
            if prob_range[j] >= p:
                offspring.append(solution_pop[j])
                break
        
    return offspring

In [15]:
#recombination and mutation
def recomb(cities, distance, solution_pop, offspring, recprob, muprob, 
           crossprob = 0.5, mulocprob = 0.5, crosstype = 'uniform', codetype = 'permutation', recomb_two_opt = False):

    pop = [s[1] for s in solution_pop] #set of decision

    new_solution_pop = []
    
    for k in range(len(offspring)//2): 

        #recombination
        if np.random.random() < recprob:
            child = []
            
            #crossover
            if crosstype == 'uniform': #uniform crossover
                for i in range(cities):
                    if np.random.random() < crossprob:
                        child.append(offspring[2*k][1][i]) #from offspring1
                    else: 
                        child.append(offspring[2*k+1][1][i]) #from offspring2
            
            elif crosstype == '1point': #1 point crossover
                cp=np.random.randint(1,cities-1)
                child = offspring[2*k][1][:cp] + offspring[2*k+1][1][cp:]
            
            elif crosstype == '2point': #2 point crossover
                cp1=np.random.randint(1,cities-2)
                cp2=np.random.randint(cp1+1,cities)
                child = offspring[2*k][1][:cp1] + offspring[2*k+1][1][cp1:cp2] + offspring[2*k][1][cp2:]
            else:
                raise ValueError("Wrong selecttype")
            
            
            #mutation: 2 different types of mutation for each coding style
            if np.random.random() < muprob:
                if codetype == 'permutation':
                    (loc1, loc2) = np.random.choice(child, size = 2, replace = False)
                    temp = child[loc1]
                    child[loc1] = child[loc2]
                    child[loc2] = temp
                    
                elif codetype == 'encode':
                    for i in range(cities):
                        if np.random.random() < mulocprob:
                            child[i] = min(cities-1-i,child[i]+1)
                        else: child[i] = max(0,child[i]-1)
                            
                else:
                    raise ValueError("Wrong codetype")


            #add to population
            if child not in pop:
                pop.append(child)
                new_solution = create_solution(distance, child, codetype, recomb_two_opt)
                if new_solution not in solution_pop:
                    new_solution_pop.append(new_solution)
            
    return new_solution_pop

In [16]:
# replacement
def replace(solution_pop, new_solution_pop, pop_size, reptype = 'best', selecttype = None):
    pop = []
    
    if reptype == 'complete':
        return new_solution_pop
    
    elif reptype == 'best':
        total_pop = solution_pop + new_solution_pop
        sortedpop = sorted(total_pop) #sorted by cost of a solution
        pop = sortedpop[0:pop_size]
        
    elif reptype == 'select':
        total_pop = solution_pop + new_solution_pop
        pop = select(total_pop, pop_size, selecttype)
    
    return pop 

In [17]:
#main algorithm
def GA_TSP (cities, distance, 
            pop_size, offspring_size, recprob, crossprob, muprob, mulocprob,
            codetype, two_opt_select, selecttype, crosstype, reptype, 
            max_iter = None, TimeLimit = None, obj_length = None):

    from time import process_time 
    start_time = process_time()      #computational time benchmarking
           
    
    solution_pop = initialize(cities, distance, pop_size, codetype = codetype, init_two_opt = two_opt_select)
    
        
    terminated = False
    it = 0 #iteration
    
    while not terminated: #termination condition    
        it += 1

        offspring = select(solution_pop, offspring_size, selecttype = selecttype)

        new_solution_pop = recomb(cities, distance, solution_pop, offspring,
                              recprob = recprob, muprob = muprob, crossprob = crossprob, mulocprob = mulocprob, 
                              crosstype = crosstype, codetype = codetype, recomb_two_opt = two_opt_select)

        solution_pop = replace(solution_pop, new_solution_pop, pop_size, reptype = reptype, selecttype = selecttype)


        sorted_solution_pop = sorted(solution_pop) #sorted by cost of a solution
        #print('Objective values of current population', [s[0] for s in sorted_solution_pop])
        
        runtime = process_time() - start_time
        #check termination condition
        if TimeLimit is not None:
            if (runtime >= TimeLimit): 
                print('Reach TimeLimit')
                terminated = True
                   
        elif obj_length is not None:
            if (sorted_solution_pop[0][0] <= obj_length):
                print('Reach objective length')
                terminated = True              
                
        elif max_iter is not None:
            if (it >= max_iter):
                print('Reach max number of iterations')
                terminated = True
                
        else:
            if (it >= 10): # default max number of iterations
                print('Reach default max number of iterations')
                terminated = True
                           
    print('Running time: ', runtime)
    best_solution = sorted_solution_pop[0]         #get the best candidate
    print('Best objective value', best_solution[0])
    #print('Decision', best_solution[1])
    
    return (best_solution[0], runtime)

In [18]:
#prepare dataset

In [19]:
# 120 cities
cities0 = 120

di = pd.read_excel("gr120.xlsx",sheet_name="DistanceMatrix")
distance0=di.values

for i in range(cities0):
    distance0[i,i] = 10000

In [20]:
# Medium size data set: # of nodes = 152
url1 = 'http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/XML-TSPLIB/instances/pr152.xml.zip'
zip1 = 'pr152.xml.zip'
file1 = 'pr152.xml'
cities1 = 152

load_data(url1, zip1)
xroot1 = import_data(file1)
distance1 = dist_matrix(cities1, xroot1)

In [21]:
# Larger size data set: # of nodes = 264
url2 = 'http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/XML-TSPLIB/instances/pr264.xml.zip'
zip2 = 'pr264.xml.zip'
file2 = 'pr264.xml'
cities2 = 264

load_data(url2, zip2)
xroot2 = import_data(file2)
distance2 = dist_matrix(cities2, xroot2)

<a id='set_params'></a>
**Setting default params**

In [22]:
max_iter_param = 500

pop_size_param = 500

offspring_size_param = 1000

recprob_param = 0.7
crossprob_param = 0.5

muprob_param = 0.5
mulocprob_param = 0.3

In [23]:
codetype_param = 'encode' #permutation
two_opt_select_param = False #True
selecttype_param = 'rank' #fitness, tournament
crosstype_param = 'uniform' #1point, 2point
reptype_param = 'best' #complete, select

In [24]:
(result, time) = GA_TSP (cities0, distance0, 
                         pop_size = pop_size_param, offspring_size = offspring_size_param, recprob = recprob_param, 
                         crossprob = crossprob_param, muprob = muprob_param, mulocprob = mulocprob_param, 
                         codetype = codetype_param, two_opt_select = two_opt_select_param, selecttype = selecttype_param, 
                         crosstype = crosstype_param, reptype = reptype_param, 
                         max_iter = max_iter_param, TimeLimit = None, obj_length = None)

Reach max number of iterations
Running time:  76.78125
Best objective value 41661


## Comparison

**Problem specific parameters**  
• Solution representation + mutation mechanism => encode (move to left) vs permutation (swap 2 points)  
• Evaluation of solutions => 2-opt vs not 2-opt  
• Design of genetic operators recombination and crossover => uniform vs 1-point crossover vs 2-point crossover  

**Generic parameters**  
• Population size  
• Number of newly generated solutions in each iteration  
• Selection mechanism => fitness vs rank vs tournament  
• Recombination and mutation probabilities  
• Replacement strategy => best out of (N+M) vs complete replacement M=N vs selection from (N+M)  
• Termination criterion => max number of iterations/TimeLimit vs objective length  


In [25]:
dataset = ((cities0, distance0), (cities1, distance1), (cities2, distance2))
index = ['Dataset 1', 'Dataset 2', 'Dataset 3']

**Notation**  
Dataset 1 = Example with 120 cities (best obj value = 6942)  
Dataset 2 = Example with 152 cities (best obj value = 73682)  
Dataset 3 = Example with 264 cities (best obj value = 49135)

### Problem specific 

#### Solution representation + mutation mechanism  
Run 2 combinations of (representation type with mutation mechanism): 1) encode by sequence position and mutation by moving to left and 2) permutation coding and mutation by swap 2 cities with 3 datasets. The other parameters are declared as above: <a href='#set_params'>*default setting*</a>. The test is run 2 times and the average result is considered for evaluation.

In [25]:
value = pd.DataFrame(index = index, columns = ['encode', 'permutation'])
codetype_set = ['encode', 'permutation']
obj_set = np.zeros([2,3,2])
time_set = np.zeros([2,3,2])

for k in range(2): #run 2 times and take the average
    for i in range(3): #dataset 
        for j in range(2): #codetype
            result = GA_TSP (dataset[i][0], dataset[i][1], 
                             pop_size = pop_size_param, offspring_size = offspring_size_param, recprob = recprob_param, 
                             crossprob = crossprob_param, muprob = muprob_param, mulocprob = mulocprob_param, 
                             codetype = codetype_set[j], two_opt_select = two_opt_select_param, selecttype = selecttype_param, 
                             crosstype = crosstype_param, reptype = reptype_param, 
                             max_iter = max_iter_param, TimeLimit = None, obj_length = None)
            obj_set[k,i,j] = result[0]
            time_set[k,i,j] = result[1]

for i in range(3): #dataset 
    for j in range(2): #codetype
        obj_mean = np.mean([obj_set[k,i,j] for k in range(2)])
        time_mean = np.mean([time_set[k,i,j] for k in range(2)])
        value.iloc[i,j] = (obj_mean, time_mean)
        

Reach max number of iterations
Running time:  41.40625
Best objective value 7512
Reach max number of iterations
Running time:  43.359375
Best objective value 9178
Reach max number of iterations
Running time:  55.390625
Best objective value 112519.63003222444
Reach max number of iterations
Running time:  47.15625
Best objective value 92185.30613250693
Reach max number of iterations
Running time:  72.03125
Best objective value 189810.84780037354
Reach max number of iterations
Running time:  70.734375
Best objective value 225305.56395576004
Reach max number of iterations
Running time:  41.5
Best objective value 8119
Reach max number of iterations
Running time:  40.15625
Best objective value 8006
Reach max number of iterations
Running time:  46.265625
Best objective value 104488.8457443189
Reach max number of iterations
Running time:  53.3125
Best objective value 83607.94205650245
Reach max number of iterations
Running time:  74.453125
Best objective value 210711.05060017365
Reach max numb

In [31]:
value1=value
print(value)

                                    encode                       permutation
Dataset 1              (7815.5, 41.453125)              (8592.0, 41.7578125)
Dataset 2  (108504.23788827166, 50.828125)    (87896.62409450469, 50.234375)
Dataset 3  (200260.9492002736, 73.2421875)  (219812.81994044723, 73.3046875)


#### Comment:   
Overall, with default setting, the performance of encode and permutation is not too much different. While the encode get slightly better results in dataset 1 and 3, the permutation is much better in dataset 2. Thee running time is nearly same between 2 approaches.

#### Evaluation of solutions   
Run 2 evaluation strategy: 1) with 2-opt (i.e. firstly find the best possible tour from original tour and build solution based on this best tour) and 2) without 2-opt (i.e. directly use the length of the tour). The other parameters are declared as above: <a href='#set_params'>*default setting*</a>. The test is run 2 times and the average result is considered for evaluation.

In [26]:
value = pd.DataFrame(index = index, columns = ['with 2-opt', 'without 2-opt'])
two_opt_select_set = [True, False]
obj_set = np.zeros([2,3,2])
time_set = np.zeros([2,3,2])

for k in range(2): #run 2 times and take the average
    for i in range(3): #dataset 
        result = GA_TSP (dataset[i][0], dataset[i][1], 
                         pop_size = 5, offspring_size = 10, recprob = recprob_param, 
                         crossprob = crossprob_param, muprob = muprob_param, mulocprob = mulocprob_param, 
                         codetype = codetype_param, two_opt_select = True, selecttype = selecttype_param, 
                         crosstype = crosstype_param, reptype = reptype_param, 
                         max_iter = None, TimeLimit = 120, obj_length = None)
        obj_set[k,i,0] = result[0]
        time_set[k,i,0] = result[1]

for k in range(2): #run 2 times and take the average
    for i in range(3): #dataset 
        result = GA_TSP (dataset[i][0], dataset[i][1], 
                         pop_size = 500, offspring_size = 1000, recprob = recprob_param, 
                         crossprob = crossprob_param, muprob = muprob_param, mulocprob = mulocprob_param, 
                         codetype = codetype_param, two_opt_select = False, selecttype = selecttype_param, 
                         crosstype = crosstype_param, reptype = reptype_param, 
                         max_iter = None, TimeLimit = 120, obj_length = None)
        obj_set[k,i,1] = result[0]
        time_set[k,i,1] = result[1]        
        
        
for i in range(3): #dataset 
    for j in range(2): #codetype
        obj_mean = np.mean([obj_set[k,i,j] for k in range(2)])
        time_mean = np.mean([time_set[k,i,j] for k in range(2)])
        value.iloc[i,j] = (obj_mean, time_mean)

Reach TimeLimit
Running time:  122.171875
Best objective value 43974
Reach TimeLimit
Running time:  131.078125
Best objective value 884225.0993515882


KeyboardInterrupt: 

In [ ]:
value2=value
print(value)

#### Comment:   
Overall, with default setting, the performance of encode and permutation is not too much different. While the encode get slightly better results in dataset 1 and 3, the permutation is much better in dataset 2. Thee running time is nearly same between 2 approaches.

#### Design of genetic operators recombination and crossover    
Run 3 crossover strategy: 1) uniform ; 2) 1-point crossover and 3) 2-point crossover. The other parameters are declared as above: <a href='#set_params'>*default setting*</a>. The test is run 2 times and the average result is considered for evaluation.

In [ ]:
value = pd.DataFrame(index = index, columns = ['uniform', '1-point crossover', '2-point crossover'])
crosstype_set = ['uniform', '1point', '2point']
obj_set = np.zeros([2,3,3])
time_set = np.zeros([2,3,3])

for k in range(2): #run 2 times and take the average
    for i in range(3): #dataset 
        for j in range(3): #crossover options
            result = GA_TSP (dataset[i][0], dataset[i][1], 
                             pop_size = pop_size_param, offspring_size = offspring_size_param, recprob = recprob_param, 
                             crossprob = crossprob_param, muprob = muprob_param, mulocprob = mulocprob_param, 
                             codetype = codetype_param, two_opt_select = two_opt_select_param, selecttype = selecttype_param, 
                             crosstype = crosstype_set[j], reptype = reptype_param, 
                             max_iter = max_iter_param, TimeLimit = None, obj_length = None)
            obj_set[k,i,j] = result[0]
            time_set[k,i,j] = result[1]

for i in range(3): #dataset 
    for j in range(3): #crossover
        obj_mean = np.mean([obj_set[k,i,j] for k in range(2)])
        time_mean = np.mean([time_set[k,i,j] for k in range(2)])
        value.iloc[i,j] = (obj_mean, time_mean)

In [ ]:
value3=value
print(value)

#### Comment:   
Overall, with default setting, the performance of encode and permutation is not too much different. While the encode get slightly better results in dataset 1 and 3, the permutation is much better in dataset 2. Thee running time is nearly same between 2 approaches.

### General parameters

#### Population size
Run tests with 4 different population sizes: 100, 250, 500, 1000 with 3 datasets. The other parameters are declared as above: <a href='#set_params'>*default setting*</a>.

In [ ]:
value = pd.DataFrame(index = index, columns = ['pop_size = 100', 'pop_size = 250', 'pop_size = 500', 'pop_size = 1000'])
pop_size_set = [100, 250, 500, 1000]

for i in range(3): #dataset 
    for j in range(4): #population size
        result = GA_TSP (dataset[i][0], dataset[i][1], 
                         pop_size = pop_size_set[j], offspring_size = offspring_size_param, recprob = recprob_param, 
                         crossprob = crossprob_param, muprob = muprob_param, mulocprob = mulocprob_param, 
                         codetype = codetype_param, two_opt_select = two_opt_select_param, selecttype = selecttype_param, 
                         crosstype = crosstype_param, reptype = reptype_param, 
                         max_iter = max_iter_param, TimeLimit = None, obj_length = None)
        value.iloc[i,j] = result



In [ ]:
value4=value
print(value)

#### Number of newly generated solutions in each iteration (offspring)
Considering the population size of 500, we run tests with 3 different offspring sizes: 500, 1000, 2000 with 3 datasets. The other parameters are declared as above: <a href='#set_params'>*default setting*</a>. 

In [ ]:
value = pd.DataFrame(index = index, columns = ['offspring_size = 500', 'offspring_size = 1000', 'offspring_size = 2000'])
offspring_size_set = [500, 1000, 2000]

for i in range(3): #dataset 
    for j in range(3): #offspring size
        result = GA_TSP (dataset[i][0], dataset[i][1], 
                         pop_size = pop_size_param, offspring_size = offspring_size_set[j], recprob = recprob_param, 
                         crossprob = crossprob_param, muprob = muprob_param, mulocprob = mulocprob_param, 
                         codetype = codetype_param, two_opt_select = two_opt_select_param, selecttype = selecttype_param, 
                         crosstype = crosstype_param, reptype = reptype_param, 
                         max_iter = max_iter_param, TimeLimit = None, obj_length = None)
        value.iloc[i,j] = result

In [ ]:
value5=value
print(value)

#### Selection mechanism
Run tests with 3 selection mechanism: 1) Fitnessproportional selection; 2) Rank-based selection and 3) Tournament selection with 3 datasets. The other parameters are declared as above: <a href='#set_params'>*default setting*</a>. 

In [ ]:
value = pd.DataFrame(index = index, columns = ['fitness', 'rank', 'tournament'])
selecttype_set = ['fitness', 'rank', 'tournament']

for i in range(3): #dataset 
    for j in range(3): #select type
        result = GA_TSP (dataset[i][0], dataset[i][1], 
                         pop_size = pop_size_param, offspring_size = offspring_size_param, recprob = recprob_param, 
                         crossprob = crossprob_param, muprob = muprob_param, mulocprob = mulocprob_param, 
                         codetype = codetype_param, two_opt_select = two_opt_select_param, selecttype = selecttype_set[j], 
                         crosstype = crosstype_param, reptype = reptype_param, 
                         max_iter = max_iter_param, TimeLimit = None, obj_length = None)
        value.iloc[i,j] = result

In [ ]:
value6=value
print(value)

#### Recombination probabilities
Run tests with 4 different recombination probabilities: 0, 0.4, 0.7, 1 with 3 datasets. The other parameters are declared as above: <a href='#set_params'>*default setting*</a>. 

In [ ]:
value = pd.DataFrame(index = index, columns = ['recombination probabiliy = 0', '= 0.4', '= 0.7', '= 1'])
recprob_set = [0, 0.4, 0.7, 1]

for i in range(3): #dataset 
    for j in range(4): #select type
        result = GA_TSP (dataset[i][0], dataset[i][1], 
                         pop_size = pop_size_param, offspring_size = offspring_size_param, recprob = recprob_set[j], 
                         crossprob = crossprob_param, muprob = muprob_param, mulocprob = mulocprob_param, 
                         codetype = codetype_param, two_opt_select = two_opt_select_param, selecttype = selecttype_param, 
                         crosstype = crosstype_param, reptype = reptype_param, 
                         max_iter = max_iter_param, TimeLimit = None, obj_length = None)
        value.iloc[i,j] = result

In [ ]:
value7=value
print(value)

#### Mutation probabilities
Run tests with 5 different mutation probabilities: 0, 0.25, 0.5, 0.75, 1 with 3 datasets. The other parameters are declared as above: <a href='#set_params'>*default setting*</a>. 

In [ ]:
value = pd.DataFrame(index = index, columns = ['recombination probabiliy = 0', '= 0.4', '= 0.7', '= 1'])
muprob_set = [0, 0.4, 0.7, 1]

for i in range(3): #dataset 
    for j in range(4): #select type
        result = GA_TSP (dataset[i][0], dataset[i][1], 
                         pop_size = pop_size_param, offspring_size = offspring_size_param, recprob = recprob_param, 
                         crossprob = crossprob_param, muprob = muprob_set[j], mulocprob = mulocprob_param, 
                         codetype = codetype_param, two_opt_select = two_opt_select_param, selecttype = selecttype_param, 
                         crosstype = crosstype_param, reptype = reptype_param, 
                         max_iter = max_iter_param, TimeLimit = None, obj_length = None)
        value.iloc[i,j] = result

In [ ]:
value8=value
print(value)

#### Replacement strategy
Run tests with 3 replacement strategies: 1) complete replacement ; 2) best N out of pool and 3) selection from pool with 3 datasets. The other parameters are declared as above: <a href='#set_params'>*default setting*</a>. 

In [ ]:
value = pd.DataFrame(index = index, columns = ['complete replacement', 'best of pool', 'selection from pool'])
reptype_set = ['complete', 'best', 'select']

for i in range(3): #dataset 
    for j in range(4): #select type
        result = GA_TSP (dataset[i][0], dataset[i][1], 
                         pop_size = pop_size_param, offspring_size = offspring_size_param, recprob = recprob_param, 
                         crossprob = crossprob_param, muprob = muprob_param, mulocprob = mulocprob_param, 
                         codetype = codetype_param, two_opt_select = two_opt_select_param, selecttype = selecttype_param, 
                         crosstype = crosstype_param, reptype = reptype_set[j], 
                         max_iter = max_iter_param, TimeLimit = None, obj_length = None)
        value.iloc[i,j] = result

#### Termination condition
Run tests with 2 termination conditions: 1) time limit/max number of iterations and 2) desired objective value. Here we will define the desired value as the optimal value of each dataset (obtained from database link) + 20% optimal value. Also the TimeLimit is set as 1 min, 2 mins and 3 mins for dataset 1, 2, 3 respectively. The other parameters are declared as above: <a href='#set_params'>*default setting*</a>. 

In [ ]:
value = pd.DataFrame(index = index, columns = ['time limit', 'desired value'])
TimeLimit_set = [60, 120, 180]
for i in range(3): #dataset 
    result = GA_TSP (dataset[i][0], dataset[i][1], 
                     pop_size = pop_size_param, offspring_size = offspring_size_param, recprob = recprob_param, 
                     crossprob = crossprob_param, muprob = muprob_param, mulocprob = mulocprob_param, 
                     codetype = codetype_param, two_opt_select = two_opt_select_param, selecttype = selecttype_param, 
                     crosstype = crosstype_param, reptype = reptype_param, 
                     max_iter = None, TimeLimit = TimeLimit_set[i], obj_length = None)
    value.iloc[i,1] = result



obj_length_set = [8400, 88800, 60000]
#desired objective value
for i in range(3): #dataset 
    result = GA_TSP (dataset[i][0], dataset[i][1], 
                     pop_size = pop_size_param, offspring_size = offspring_size_param, recprob = recprob_param, 
                     crossprob = crossprob_param, muprob = muprob_param, mulocprob = mulocprob_param, 
                     codetype = codetype_param, two_opt_select = two_opt_select_param, selecttype = selecttype_param, 
                     crosstype = crosstype_param, reptype = reptype_param, 
                     max_iter = None, TimeLimit = None, obj_length = obj_length_set[i])
    value.iloc[i,1] = result